In [1]:
from unified_planning.shortcuts import *
from unified_planning.plans import *
from importlib import reload

In [2]:
import unifiedplanning_blocksworld as bw
import prompts
import planbench as pb

In [3]:
reload(bw)
reload(pb)

<module 'planbench' from '/Users/matthiasgrabmair/git/planning-with-llms/src/planbench.py'>

In [4]:
p = bw.BlocksworldProblem()

In [5]:
pb.parse_planbench_initial_condition(p, prompts.gen_problem_init)
pb.parse_planbench_goal_state(p, prompts.gen_problem_goal)

In [6]:
p.generate_plan()

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 94 of `/Users/matthiasgrabmair/git/planning-with-llms/src/unifiedplanning_blocksworld.py`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.

##########Pyperplan returned: SequentialPlan:
    unstack(blue, orange)
    stack(blue, yellow)
    unstack(orange, red)
    put_down(orange)
    pick_up(red)
    stack(red, orange)############
########PlanKind.SEQUENTIAL_PLAN#######


/Users/matthiasgrabmair/miniconda3/envs/base311/lib/python3.11/site-packages/unified_planning/engines/mixins/oneshot_planner.py:76: UserWarning: We cannot establish whether Pyperplan can solve this problem!
  warn(msg)


PlanGenerationResult(status=<PlanGenerationResultStatus.SOLVED_SATISFICING: 1>, plan=SequentialPlan([unstack(blue, orange), stack(blue, yellow), unstack(orange, red), put_down(orange), pick_up(red), stack(red, orange)]), engine_name='Pyperplan', metrics={'engine_internal_time': '0.002317190170288086'}, log_messages=None)